In [1]:
import pickle
import os
import csv
from collections import Counter, defaultdict
from constants import COST_COLOR_ORDER, COLOR_ORDER

In [2]:
with open('run7_comparison_game_data.dat', 'rb') as f:
    game_data = pickle.load(f)
    
with open('run7_comparison_win_data.dat', 'rb') as f:
    win_data = pickle.load(f)

In [3]:
csv_rows = []

In [4]:
# base level, other_level, base player pid
game_data.keys()

dict_keys([(11, 11, 3), (5, 1, 2), (1, 5, 3), (1, 11, 3), (11, 11, 2), (5, 11, 3), (1, 5, 2), (1, 11, 2), (11, 1, 3), (5, 11, 2), (5, 5, 1), (1, 11, 1), (11, 1, 2), (5, 11, 1), (5, 5, 0), (1, 1, 1), (1, 11, 0), (11, 1, 1), (5, 11, 0), (5, 5, 3), (1, 1, 0), (11, 1, 0), (5, 5, 2), (1, 1, 3), (11, 5, 3), (5, 1, 1), (1, 1, 2), (11, 5, 2), (11, 11, 1), (5, 1, 0), (1, 5, 1), (11, 5, 1), (11, 11, 0), (5, 1, 3), (1, 5, 0), (11, 5, 0)])

In [5]:
print(win_data[(1,1,1)])
print(game_data[(1,1,1)][0])

Counter({3: 17, 0: 13, 2: 13, 1: 7})
{'game': {'gems': ColorCombination object: 
{
    "black": 5,
    "red": 4,
    "white": 3,
    "green": 4,
    "blue": 2,
    "gold": 5
}, 'player_order': None, 'id': 451, 'start_time': datetime.datetime(2018, 4, 25, 10, 48, 35, 911642), 'available_cards': {'tier 1': [{'color': 'black', 'points': 0, 'cost_': {'white': 0, 'black': 0, 'blue': 0, 'green': 1, 'red': 3}, 'tier': 1, 'cost': ColorCombination object: 
{
    "white": 0,
    "black": 0,
    "blue": 0,
    "green": 1,
    "red": 3
}}, {'color': 'white', 'points': 1, 'cost_': {'white': 0, 'black': 0, 'blue': 0, 'green': 4, 'red': 0}, 'tier': 1, 'cost': ColorCombination object: 
{
    "white": 0,
    "black": 0,
    "blue": 0,
    "green": 4,
    "red": 0
}}, {'color': 'green', 'points': 0, 'cost_': {'white': 0, 'black': 2, 'blue': 1, 'green': 0, 'red': 2}, 'tier': 1, 'cost': ColorCombination object: 
{
    "white": 0,
    "black": 2,
    "blue": 1,
    "green": 0,
    "red": 2
}}, {'color': 'b

In [6]:
TIER_POINTS = {1: [0, 1], 2: [1,2,3], 3:[3,4,5]}
TIER_COSTS = {1: [3,4,5], 2: [5,6,7,8], 3:[7,10,14,12,]}

def parse_player_data(data, pid):
    return_data = {}
    for tier in [1,2,3]:
        for reserve_type in ['topdeck','board']:
            return_data['player_{pid}_{tier}_reserve_{reserve_type}'.format(**locals())] = (
                data['total_times_reserved'][(tier, reserve_type)]
            )
    return_data['player_{pid}_win'.format(pid=pid)] = data['win'] * 1
    return_data['player_{pid}_n_reserved_cards'.format(pid=pid)] = data['n_reserved_cards']
    return_data['player_{pid}_points'.format(pid=pid)] = data['points'] 
    return_data['player_{pid}_n_cards'.format(pid=pid)] = data['n_cards']
    
    # gems taken
    for gem_amount in [-2,-1,0,1,2,3]:
        return_data['player_{pid}_{gem_amount}_gems_taken'.format(**locals())] = (
            data['total_gems_taken'][gem_amount]
        )
    
    # card colors
    color_counter = Counter()
    tier_counter = Counter()
    points_counter = defaultdict(Counter)
    cost_counter = defaultdict(Counter)
    for card in data['cards']:
        color_counter[card['color']] += 1
        tier_counter[card['tier']] += 1
        points_counter[card['tier']][card['points']] += 1
        cost_counter[card['tier']][card['cost'].count()] += 1
        
    for color in COST_COLOR_ORDER:
        return_data['player_{pid}_ncards_{color}'.format(**locals())] = color_counter[color]
        
    for tier in [1,2,3]:
        return_data['player_{pid}_ncards_tier_{tier}'.format(**locals())] = tier_counter[tier]
        for tier_points in TIER_POINTS[tier]:
            return_data['player_{pid}_points_tier_{tier}_points_is_{tier_points}'.format(**locals())] = (
                points_counter[tier][tier_points]
            )
        for tier_costs in TIER_COSTS:
            return_data['player_{pid}_costs_tier_{tier}_cost_is_{tier_costs}'.format(**locals())] = (
                cost_counter[tier][tier_costs]
            )
        
    # count objectives
    return_data['player_{pid}_n_objectives'.format(pid=pid)] = len(data['objectives'])

    # record order
    return_data['player_{pid}_order'.format(pid=pid)] = data['order']
    return return_data

def parse_game_data(data, tuple_key):
    player_data = data['players']
    row_data = {}
    row_data['game_id'] = data['game']['id']
    row_data['n_turns'] = data['game']['turn']
    row_data['n_rounds'] = data['game']['round']
    for pid in range(4):
        row_data.update(parse_player_data(player_data[pid], pid))
        if pid==tuple_key[2]:
            row_data['player_{pid}_ai_level'.format(pid=pid)] = tuple_key[0]
            row_data['unique_player_win'] = row_data['player_{pid}_win'.format(pid=pid)]
        else:
            row_data['player_{pid}_ai_level'.format(pid=pid)] = tuple_key[1]
        if row_data['player_{pid}_win'.format(pid=pid)] == 1:
            row_data['player_winner_n_objectives'] = row_data['player_{pid}_n_objectives'.format(pid=pid)]
            row_data['player_winner_ncards_tier_1'] = row_data['player_{pid}_ncards_tier_1'.format(pid=pid)]
            row_data['player_winner_ncards_tier_2'] = row_data['player_{pid}_ncards_tier_2'.format(pid=pid)]
            row_data['player_winner_ncards_tier_3'] = row_data['player_{pid}_ncards_tier_3'.format(pid=pid)]
    row_data['unique_player_id'] = tuple_key[2]
    row_data['unique_player_level'] = tuple_key[0]
    row_data['other_player_level'] = tuple_key[1]
    return row_data
                

In [7]:
for tuple_key in game_data.keys():
    for data in game_data[tuple_key]:
        csv_rows.append(parse_game_data(data, tuple_key))
        
row_header = list(csv_rows[0].keys())


In [8]:
with open('splendor_comparison_data.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=sorted(row_header))
    writer.writeheader()
    for row in csv_rows:
        writer.writerow(row)